In [6]:
import json
import time 

import pandas as pd

from kafka import KafkaProducer

In [2]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

In [3]:
producer.bootstrap_connected()

True

In [5]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

t1 = time.time()
producer.flush()

t2 = time.time()
print(f'send took {(t1 - t0):.2f} seconds')
print(f'flush took {(t2 - t1):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
send took 0.51 seconds
flush took 0.00 seconds


In [13]:
df_green = pd.read_csv('green_tripdata_2019-10.csv.gz', usecols=['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 
'passenger_count', 'trip_distance', 'tip_amount'])
# TypeError: Object of type Timestamp is not JSON serializable
# df_green.lpep_pickup_datetime = pd.to_datetime(df_green.lpep_pickup_datetime)
# df_green.lpep_dropoff_datetime = pd.to_datetime(df_green.lpep_dropoff_datetime)

In [14]:
df_green.shape

(476386, 7)

In [15]:
t0 = time.time()

topic_name = 'green-trips'

for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(topic_name, value=row_dict)


producer.flush()

t2 = time.time()
print(f'send took {(t2 - t0):.2f} seconds')

send took 169.45 seconds


In [16]:
import pyspark
from pyspark.sql import SparkSession

In [20]:
pyspark_version = pyspark.__version__
pyspark_version

'3.5.1'

In [21]:
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

In [23]:
%env JAVA_HOME=/home/ale/prg/py/dezoomcamp/hw5/spark/jdk-11.0.2

env: JAVA_HOME=/home/ale/prg/py/dezoomcamp/hw5/spark/jdk-11.0.2


In [24]:
%env JAVA_HOME

'/home/ale/prg/py/dezoomcamp/hw5/spark/jdk-11.0.2'

In [25]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/04/07 21:13:39 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 172.17.101.240 instead (on interface eth0)
24/04/07 21:13:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ale/prg/py/dezoomcamp/venv/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ale/.ivy2/cache
The jars for the packages stored in: /home/ale/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e12521c3-a237-4aa9-bef1-8b868ddeeefb;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-sq

In [26]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [27]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

In [28]:
query = green_stream.writeStream.foreachBatch(peek).start()

24/04/07 21:17:44 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-1398fed7-b440-42a1-8ec2-d1eed9228532. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/07 21:17:44 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/04/07 21:17:46 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


Row(key=None, value=bytearray(b'{"lpep_pickup_datetime": "2019-10-01 00:26:02", "lpep_dropoff_datetime": "2019-10-01 00:39:58", "PULocationID": 112, "DOLocationID": 196, "passenger_count": 1.0, "trip_distance": 5.88, "tip_amount": 0.0}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 4, 7, 20, 51, 5, 184000), timestampType=0)


In [30]:
query.status

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

In [31]:
query.stop()

In [32]:
from pyspark.sql import types

In [33]:
schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [34]:
from pyspark.sql import functions as F

green_stream = green_stream.select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")).select("data.*")

In [35]:
green_stream

DataFrame[lpep_pickup_datetime: string, lpep_dropoff_datetime: string, PULocationID: int, DOLocationID: int, passenger_count: double, trip_distance: double, tip_amount: double]